### Import functions

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import radians, cos, sin, asin, sqrt
from sklearn.svm import OneClassSVM
from functionsPredictions import *
from sklearn import preprocessing
import matplotlib.pyplot as plt
import category_encoders as ce
from datetime import timedelta
from scipy import stats
import seaborn as sns
import xgboost as xgb
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import glob

warnings.filterwarnings(action='ignore')
plt.style.use('seaborn-poster')
sns.set_context("poster") 

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_holiday2)

### Reading in filepaths

In [2]:
dataframe = getDataframe()
dataframe.head()

,Location,Visits,Date,Time,Journeys,Windspeed,Temperature,Clouds,Rain amount,Rain duration,...,Holiday_Name_Summer holiday,Holiday_Name_Whit,Holiday_Name_Winter holiday,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,stringency_index
0,Erasmuspark,243.0,2020-10-01,00:15:00,91.807802,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
1,Oosterpark,53.0,2020-10-01,00:15:00,186.450812,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
2,Sarphatipark,96.0,2020-10-01,00:15:00,146.362942,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
3,Vondelpark West,80.0,2020-10-01,00:15:00,16.557784,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
4,Westergasfabriek,35.0,2020-10-01,00:15:00,14.840649,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04


In [3]:
dataframe_no_outliers = removeOutliers(dataframe)
dataframe_no_outliers.head()

,Location,Visits,Date,Time,Journeys,Windspeed,Temperature,Clouds,Rain amount,Rain duration,...,Holiday_Name_Summer holiday,Holiday_Name_Whit,Holiday_Name_Winter holiday,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,stringency_index
Datetime,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:15:00,Erasmuspark,243.0,2020-10-01,00:15:00,91.807802,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
2020-10-01 00:15:00,Oosterpark,53.0,2020-10-01,00:15:00,186.450812,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
2020-10-01 00:15:00,Sarphatipark,96.0,2020-10-01,00:15:00,146.362942,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
2020-10-01 00:15:00,Vondelpark West,80.0,2020-10-01,00:15:00,16.557784,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04
2020-10-01 00:15:00,Westergasfabriek,35.0,2020-10-01,00:15:00,14.840649,5.0,15.575,8.0,0.0,0.0,...,0,0,0,-47.0,-20.0,-43.0,-55.0,-43.0,16.0,62.04


In [4]:
dataframe_no_outliers.to_csv('data_preprocessing.csv')

### Make predictions

In [ ]:
df = pd.read_csv('data_preprocessing.csv')

df["Date"] = df["Date"].astype('str')
df['Datetime']=pd.to_datetime(df.Date + ' ' + df.Time, format='%Y/%m/%d %H:%M:%S')

In [ ]:
data_aug = df.copy()

In [ ]:
data_aug = add_time_vars(data_aug, onehot=True)
data_aug.drop(['Year', 'Month', 'Day_x', 'Weekday', 'Hour', 'Minute'], inplace=True, axis=1)

In [ ]:
predictor_cols = data_aug.columns.to_list()[5:]
predictions, pred = predict(data_aug, 'Erasmuspark', predictor_cols, 1000)